<a href="https://colab.research.google.com/github/hcuksuresh/BERT-pytorch/blob/master/text_clf_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://www.analyticsvidhya.com/blog/2018/11/tutorial-text-classification-ulmfit-fastai-library/#comment-155947
print('hello world')

hello world


In [1]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
    100% |████████████████████████████████| 576.6MB 29kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x62846000 @  0x7f64fc2242a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 133kB 7.7MB/s 
    100% |████████████████████████████████| 61kB 21.8MB/s 
    100% |████████████████████████████████| 112kB 20.7MB/s 
    100% |████████████████████████████████| 163kB 19.6MB/s 
  Running setup.py bdist_wheel for bottleneck ... - \ | / - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/f2/bf/ec/e0f39aa27001525ad455139ee57ec7d0776fe074dfd78c97e4
Successfully built bottleneck
torchvision-nightly 0.2.1 has requirement pillow>=

In [0]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [3]:
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [0]:
df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})

In [21]:
df.head()

,label,text
0,17,Well i'm not sure about the story nad it did s...
1,0,"\n\n\n\n\n\n\nYeah, do you expect people to re..."
2,17,Although I realize that principle is not one o...
3,11,Notwithstanding all the legitimate fuss about ...
4,10,"Well, I will have to change the scoring on my ..."


In [22]:
df.shape

(11314, 2)

In [24]:
df.label.unique()


array([17,  0, 11, 10, 15,  4, 13, 12,  1,  6, 19, 16,  8,  7,  2,  5,  3, 14,  9, 18])

In [0]:
#slecting 1 and 10 categories only out of 20
df = df[df['label'].isin([1,10])]
df = df.reset_index(drop = True)

In [26]:
df.head()

,label,text
0,10,"Well, I will have to change the scoring on my ..."
1,1,Archive-name: graphics/resources-list/part1\nL...
2,10,"\nAnd of course, Mike Ramsey was (at one time)..."
3,10,"As I promised, I would give you the name of th..."
4,10,GAME(S) OF 4/15\n---------------\nADIRONDACK 6...


In [27]:
df['label'].value_counts()

10    600
1     584
Name: label, dtype: int64

Data Preprocessing

In [0]:
df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")

In [29]:
df.head()

,label,text
0,10,Well I will have to change the scoring on my ...
1,1,Archive name graphics resources list part La...
2,10,And of course Mike Ramsey was at one time ...
3,10,As I promised I would give you the name of th...
4,10,GAME S OF ADIRONDACK C...


In [30]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

In [0]:
# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split())


In [0]:
# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])


In [0]:
# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['text'] = detokenized_doc

In [38]:
df.head()

,label,text
0,10,Well I change scoring playoff pool Unfortunate...
1,1,Archive name graphics resources list part Last...
2,10,And course Mike Ramsey one time captain Buffal...
3,10,As I promised I would give name Panther presid...
4,10,GAME S OF ADIRONDACK CDI Adirondack leads seri...


In [0]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.4, random_state = 12)

In [40]:
df_trn.head()

,label,text
1018,10,It looks like Edmonton Oilers decided take Eur...
762,10,This kills Speaking die hard I I read died har...
768,1,The idea clip one polygon using another polygo...
152,10,I Edmonton usually least OFTEN case treated ac...
426,10,You know absolutely right I think round player...


In [41]:
df_trn.shape, df_val.shape

((710, 2), (474, 2))

In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

In [0]:
# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

**Fine-Tuning the Pre-Trained Model and Making predictions**


In [0]:
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.7)

In [45]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,7.826955,6.315040,0.143289


In [0]:
learn.save_encoder('ft_enc')

In [0]:
learn = text_classifier_learner(data_clas, drop_mult=0.7)
learn.load_encoder('ft_enc')

In [48]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.523157,0.283555,0.909283


In [0]:
# get predictions (for the validation set which is there in learner object)
preds, targets = learn.get_preds()


In [0]:
predictions = np.argmax(preds, axis = 1)
predictions_0 = np.argmax(preds, axis = 0)

In [113]:
predictions_0

tensor([32,  1])

In [0]:
pred_evl = pd.DataFrame()

pred_evl['predictions'] = predictions

In [0]:
pred_evl['preds'] = preds

In [115]:
pred_evl.head(10)

,predictions,preds
0,0,"(tensor(0.9848), tensor(0.0152))"
1,1,"(tensor(0.0043), tensor(0.9957))"
2,0,"(tensor(0.5970), tensor(0.4030))"
3,1,"(tensor(0.1725), tensor(0.8275))"
4,0,"(tensor(0.7184), tensor(0.2816))"
5,0,"(tensor(0.5340), tensor(0.4660))"
6,1,"(tensor(0.0394), tensor(0.9606))"
7,1,"(tensor(0.0885), tensor(0.9115))"
8,1,"(tensor(0.1491), tensor(0.8509))"
9,1,"(tensor(0.2464), tensor(0.7536))"


In [55]:
pd.crosstab(predictions, targets)

col_0,0,1
row_0,,
0,209,18
1,25,222


In [58]:
df_val.head(1)

,label,text
490,1,I looking source code radiosity method I know ...


In [80]:
#df_val['text'].head(1)
df_val.head(5)

,label,text
490,1,I looking source code radiosity method I know ...
1102,1,I interested information stereoscopic imaging ...
557,10,Does anyone NHL STANDINGS March th I need IMME...
209,10,A little supplement Basic Mike info For Sundda...
774,10,World Championships Germany Group A results SW...


In [93]:
learn.predict(str('Does anyone NHL STANDINGS March th I need IMME'))
#new_all_pred = np.argmax(new_pred, axis = 1)

(Category 1, tensor(0), tensor([0.5504, 0.4496]))

In [90]:
new_pred

(Category 10, tensor(1), tensor([0.3146, 0.6854]))

In [76]:
learn.predict(str(df_val['text'].head(2)))

(Category 1, tensor(0), tensor([0.9265, 0.0735]))

In [82]:
learn.predict(str(df_val['text'].head(5)))

(Category 1, tensor(0), tensor([0.7942, 0.2058]))